<a href="https://colab.research.google.com/github/hdauphin/PHYS420Fall25/blob/main/HW04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# HW 4 - Integration

import numpy as np
import matplotlib.pyplot as plt

# 1. Code that calculates an integral using one of the Newton-Cotes rules (n>=2), such as Simpson's rule or higher

def simpson(f, a, b, n):
  """
  Integrate function from a to b using Simpson's rule
  Args:
    f: function
    a: lower limit
    b: upper limit
    n: number of intervals
  Returns:
    Estimate of integral
  """

  # exceptions
  if n%2!=0:  # ensure n is even
    n = n+1

  if a==b:  # return 0 if a equals b
    return 0

  if a>b:  # flip limits and negate result if a>b
    return -1*simpson(f, b, a, n)

  # body of function
  h = ((b-a)/n) # width of intervals
  x0 = a
  xn = b
  y0 = f(a)
  yn = f(b)
  x_i = np.linspace(a, b, n+1)
  f_i = f(x_i)
  endpoints = y0 + yn
  oddSum = np.sum(f_i[1:n:2])
  evenSum = np.sum(f_i[2:n-1:2])
  return (h/3)*(endpoints + 4*oddSum + 2*evenSum)

def f1(x):
  return x**3

def f2(x):
  return np.e ** x

def f3(x):
  return np.sin(x)


In [33]:
estimate1 = simpson(f1, 0, 1, 100)  # analytical answer = 1/4
print(estimate1)

estimate2 = simpson(f2, 0, 1, 100)  # analytical answer = e - 1
print(estimate2)

estimate3 = simpson(f3, 0, np.pi, 100)  # analytical answer = 2
print(estimate3)


0.25
1.7182818285545043
2.000000010824504
